In [ ]:
# Function to clear GPU cache
def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [ ]:
# Install necessary packages
!pip install openai==0.28
!pip install datasets
!pip install transformers

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import openai
import random
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoConfig
from collections import Counter
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires f

In [ ]:
# Set up API key for OpenAI
openai.api_key = 'insert-your-api-key-here'

**SST-2**

In [ ]:
# Load SST-2 dataset from GLUE
sst2_data = load_dataset("glue", "sst2")
train_data = [{"text": item["sentence"], "label": item["label"]} for item in sst2_data['train']]
test_data = [{"text": item["sentence"], "label": item["label"]} for item in sst2_data['validation']]

# Define plug-in model and tokenizer
plugin_model_name = "roberta-base"
plugin_model = AutoModelForSequenceClassification.from_pretrained(plugin_model_name)
plugin_tokenizer = AutoTokenizer.from_pretrained(plugin_model_name)

# Move the plug-in model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
plugin_model.to(device)

# Clear cache to free up GPU memory to prevent potential crashes
def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Prepare the dataset for the Trainer
class SST2Dataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer(item['text'], truncation=True, padding="max_length", max_length=128)
        encoding['labels'] = item['label']
        return {key: torch.tensor(val) for key, val in encoding.items()}

train_dataset = SST2Dataset(train_data, plugin_tokenizer)
test_dataset = SST2Dataset(test_data, plugin_tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./sst2_finetuned_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer for fine-tuning
trainer = Trainer(
    model=plugin_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
plugin_model.save_pretrained("./sst2_finetuned_model")
plugin_tokenizer.save_pretrained("./sst2_finetuned_model")

# Load the fine-tuned model and tokenizer for SuperICL
plugin_model = AutoModelForSequenceClassification.from_pretrained("./sst2_finetuned_model").to(device)
plugin_tokenizer = AutoTokenizer.from_pretrained("./sst2_finetuned_model")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.244200,0.244188
2,0.159300,0.282187
3,0.163200,0.326703


In [ ]:
# Define the LLM API call (OpenAI's GPT-4)
def call_llm(input_text):
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Updated to GPT-4
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": input_text}
        ],
        max_tokens=100,
        temperature=0
    )
    return response['choices'][0]['message']['content'].strip()

In [ ]:
# Get plug-in model predictions
def get_plugin_predictions(texts):
    clear_gpu_cache()
    inputs = plugin_tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = plugin_model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    result = []
    for prediction in predictions:
        confidence, label_id = torch.max(prediction, dim=-1)
        label = plugin_model.config.id2label[label_id.item()]
        result.append((label, confidence.item()))
    return result

In [ ]:
# Construct context string with in-context examples, labels, and confidence scores
def construct_context(examples, plugin_predictions):
    context = "Fine-tuning Plug-in Model: The RoBERTa model fine-tuned on SST-2 for sentiment classification.\n\n"
    for i, (text, (label, confidence)) in enumerate(zip(examples, plugin_predictions)):
        context += f"Example {i+1}:\nText: {text}\nPrediction: {label} (Confidence: {confidence:.2f})\n\n"
    return context


In [ ]:
def supericl_predict(test_input, context, plugin_prediction):
    plugin_label, plugin_confidence = plugin_prediction
    prompt = (f"{context}\nTest Input: {test_input}\n"
              f"The task is to classify the sentiment as 'positive' or 'negative' based on movie reviews.\n"
              f"Plug-in Prediction: {plugin_label} (Confidence: {plugin_confidence:.2f})\n"
              f"Does this label correctly match the sentiment in the test input? If not, please provide the correct label and explain the adjustment.")

    llm_output = call_llm(prompt)
    final_prediction = plugin_label
    final_confidence = plugin_confidence

    if "Prediction:" in llm_output:
        final_prediction = llm_output.split("Prediction:")[-1].strip().split()[0]
    if "Confidence:" in llm_output:
        try:
            final_confidence = float(llm_output.split("Confidence:")[-1].strip().split()[0])
        except ValueError:
            final_confidence = plugin_confidence

    explanation = (f"LLM overrides the plug-in prediction from {plugin_label} to {final_prediction} "
                   f"with a confidence of {final_confidence:.2f}."
                   if final_prediction != plugin_label else
                   f"LLM agrees with the plug-in prediction at a confidence level of {final_confidence:.2f}.")

    return final_prediction, final_confidence, explanation


In [ ]:
# 1. Function to select a balanced sample of in-context examples with high-confidence predictions
def select_balanced_examples(data, size):
    positive_examples = [example for example in data if example['label'] == 1]
    negative_examples = [example for example in data if example['label'] == 0]
    sample_size = min(size // 2, len(positive_examples), len(negative_examples))

    # Randomly sample from positive and negative examples to create a balanced in-context set
    in_context_examples = random.sample(positive_examples, sample_size) + random.sample(negative_examples, sample_size)
    return in_context_examples

In [ ]:
def run_supericl(test_data, training_data):
    in_context_examples = select_balanced_examples(training_data, sample_size)
    in_context_texts = [example['text'] for example in in_context_examples]
    plugin_predictions = get_plugin_predictions(in_context_texts)
    context = construct_context(in_context_texts, plugin_predictions)

    results = []
    for test_example in test_data:
        plugin_prediction = get_plugin_predictions([test_example])[0]
        final_prediction, final_confidence, explanation = supericl_predict(test_example, context, plugin_prediction)
        results.append({
            "test_input": test_example,
            "plugin_prediction": plugin_prediction,
            "final_prediction": final_prediction,
            "final_confidence": final_confidence,
            "explanation": explanation
        })
    return results

In [ ]:
# Use a small subset of test data to check results quickly
test_data_subset = [example["text"] for example in test_data[:30]]

# Define the number of in-context examples
sample_size = 10  # Adjust based on available resources and desired context size

# Run SuperICL
results = run_supericl(test_data_subset, train_data)

# Display results
for result in results:
    print("Test Input:", result["test_input"])
    print("Plug-in Prediction:", result["plugin_prediction"])
    print("Final Prediction:", result["final_prediction"])
    print("Explanation:", result["explanation"])
    print()


Test Input: it 's a charming and often affecting journey . 
Plug-in Prediction: ('LABEL_1', 0.9998151659965515)
Final Prediction: LABEL_1
Explanation: LLM agrees with the plug-in prediction at a confidence level of 1.00.

Test Input: unflinchingly bleak and desperate 
Plug-in Prediction: ('LABEL_0', 0.9988759160041809)
Final Prediction: LABEL_0
Explanation: LLM agrees with the plug-in prediction at a confidence level of 1.00.

Test Input: allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . 
Plug-in Prediction: ('LABEL_1', 0.9997974038124084)
Final Prediction: LABEL_1
Explanation: LLM agrees with the plug-in prediction at a confidence level of 1.00.

Test Input: the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . 
Plug-in Prediction: ('LABEL_1', 0.9997885823249817)
Final Prediction: LABEL_1
Explanation: LLM agrees with the plug-in prediction at a confidence level of 1.

In [ ]:
from sklearn.metrics import accuracy_score

# Extract true labels from the SST-2 test data subset (first 30 examples)
true_labels = [example["label"] for example in test_data[:30]]

# Extract final predictions from the results of run_supericl
# Convert predictions to numeric labels for comparison (1 = positive, 0 = negative)
final_predictions = [1 if result["final_prediction"].lower() == "positive" else 0 for result in results]

# Calculate accuracy
accuracy = accuracy_score(true_labels, final_predictions)
print("Accuracy for SST-2 (SuperICL):", accuracy)



Accuracy for SST-2 (SuperICL): 0.5666666666666667


**MNLI**

In [ ]:
# Load the MNLI dataset from GLUE
mnli_data = load_dataset("glue", "mnli")
train_data = [{"premise": item["premise"], "hypothesis": item["hypothesis"], "label": item["label"]} for item in mnli_data['train']]
test_data = [{"premise": item["premise"], "hypothesis": item["hypothesis"], "label": item["label"]} for item in mnli_data['validation_matched']]

# Define plug-in model (using roberta-base for faster performance) and tokenizer
plugin_model_name = "roberta-base"
config = AutoConfig.from_pretrained(plugin_model_name, output_hidden_states=True, num_labels=3)  # Set num_labels in config
plugin_model = AutoModelForSequenceClassification.from_pretrained(plugin_model_name, config=config)
plugin_tokenizer = AutoTokenizer.from_pretrained(plugin_model_name)

# Move the plug-in model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
plugin_model.to(device)

# Function to clear GPU cache
def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define the LLM API call (OpenAI's GPT-3 or GPT-4)
def call_llm(input_text):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": input_text}
        ],
        max_tokens=100,
        temperature=0
    )
    return response['choices'][0]['message']['content'].strip()

In [ ]:
# Fine-tune the plug-in model on a larger subset with more epochs to improve learning
def fine_tune_plugin_model(train_data, model, tokenizer):
    # Prepare dataset for fine-tuning
    train_texts = [f"Premise: {ex['premise']} Hypothesis: {ex['hypothesis']}" for ex in train_data]
    train_labels = [ex['label'] for ex in train_data]

    # Tokenize the training data
    train_encodings = tokenizer(train_texts, truncation=True, padding=True)
    train_labels_tensor = torch.tensor(train_labels)

    # Create Dataset object for training
    class MnliDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels
        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item["labels"] = self.labels[idx]
            return item
        def __len__(self):
            return len(self.labels)

    train_dataset = MnliDataset(train_encodings, train_labels_tensor)

    # Increase number of epochs and adjust batch size based on available memory
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=4,  # Adjust this based on memory constraints
        num_train_epochs=3,  # Increased to improve learning
        logging_dir="./logs",
        logging_steps=10,
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )

    # Fine-tune the model
    trainer.train()
    print("Fine-tuning complete.")

In [ ]:
# Function to get predictions with confidence scores from the plug-in model
def get_plugin_predictions(texts):
    inputs = plugin_tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = plugin_model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    result = []
    for prediction in predictions:
        confidence, label_id = torch.max(prediction, dim=-1)
        label = plugin_model.config.id2label[label_id.item()]
        result.append((label, confidence.item()))
    return result

In [ ]:
# Function to clear GPU cache
def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


In [ ]:
# Function to get embeddings from the plugin model
def get_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the CLS token representation as a sentence embedding from the last hidden layer
    embedding = outputs.hidden_states[-1][:, 0, :].cpu().numpy()  # Extract CLS token
    return embedding

# Dynamic selection of in-context examples based on similarity using embeddings
def select_in_context_examples(test_premise, test_hypothesis, training_data, tokenizer, model, num_examples=2):
    test_text = f"Premise: {test_premise} Hypothesis: {test_hypothesis}"
    test_embedding = get_embedding(test_text, tokenizer, model)

    # Compute cosine similarity between the test embedding and each training example embedding
    similarities = []
    for ex in training_data:
        train_text = f"Premise: {ex['premise']} Hypothesis: {ex['hypothesis']}"
        train_embedding = get_embedding(train_text, tokenizer, model)
        similarity = cosine_similarity(test_embedding, train_embedding)
        similarities.append(similarity[0][0])

    # Sort by similarity and select top examples
    sorted_indices = sorted(range(len(similarities)), key=lambda k: similarities[k], reverse=True)
    selected_examples = [training_data[i] for i in sorted_indices[:num_examples]]

    in_context_texts = [f"Premise: {ex['premise']} Hypothesis: {ex['hypothesis']}" for ex in selected_examples]
    plugin_predictions = get_plugin_predictions(in_context_texts)

    return construct_context(in_context_texts, plugin_predictions)


In [ ]:
# Construct context with in-context examples, labels, and confidence scores
def construct_context(examples, plugin_predictions):
    context = ""
    for i, (text, (label, confidence)) in enumerate(zip(examples, plugin_predictions)):
        context += f"Example {i+1}:\n{text}\nPrediction: {label} (Confidence: {confidence:.2f})\n\n"
    return context

In [ ]:
# Predict function with LLM and plug-in integration, with a higher confidence threshold
def supericl_predict_mnli(premise, hypothesis, context, plugin_prediction, confidence_threshold=0.7):
    plugin_label, plugin_confidence = plugin_prediction
    override_llm = plugin_confidence < confidence_threshold

    # Enhanced prompt if confidence is below threshold
    prompt = (f"{context}\nPremise: {premise}\nHypothesis: {hypothesis}\n"
              f"Plug-in Prediction: {plugin_label} (Confidence: {plugin_confidence:.2f})\n"
              f"{'The plug-in model seems uncertain.' if override_llm else ''} "
              "Does the LLM agree with the plug-in prediction? If not, provide only the final prediction label (e.g., 'LABEL_0' or 'LABEL_1').")

    llm_output = call_llm(prompt)

    # Override prediction only if confidence is below threshold and LLM suggests an alternate label
    final_prediction = plugin_label
    if override_llm and "LABEL_" in llm_output:
        final_prediction = llm_output.split("LABEL_")[-1].strip().split()[0]
        final_prediction = f"LABEL_{final_prediction}" if final_prediction.isdigit() else plugin_label

    explanation = (f"LLM overrides the plug-in prediction from {plugin_label} to {final_prediction} based on low confidence."
                   if final_prediction != plugin_label else "LLM agrees with the plug-in prediction.")
    return final_prediction, explanation

In [ ]:
# Run predictions on the test dataset in smaller batches to save memory
def run_supericl_mnli(test_data, training_data):
    results = []
    for example in test_data:
        premise = example["premise"]
        hypothesis = example["hypothesis"]

        # Select dynamic in-context examples using the plugin_model for embeddings
        context = select_in_context_examples(premise, hypothesis, training_data, plugin_tokenizer, plugin_model)

        # Get plug-in prediction
        plugin_prediction = get_plugin_predictions([f"Premise: {premise} Hypothesis: {hypothesis}"])[0]
        clear_gpu_cache()  # Clear GPU cache after each prediction batch

        # Perform final prediction with LLM override logic
        final_prediction, explanation = supericl_predict_mnli(premise, hypothesis, context, plugin_prediction)

        results.append({
            "premise": premise,
            "hypothesis": hypothesis,
            "label": example["label"],
            "plugin_prediction": plugin_prediction,
            "final_prediction": final_prediction,
            "explanation": explanation
        })
    return results


In [ ]:
# Fine-tune the plug-in model on an increased subset of data with more epochs
train_data_subset = train_data[:2000]  # Increase subset size to improve learning
fine_tune_plugin_model(train_data_subset, plugin_model, plugin_tokenizer)
clear_gpu_cache()  # Clear GPU cache after fine-tuning

# Run SuperICL predictions on a subset of test data in batches
test_data_subset = test_data[:10]
results = run_supericl_mnli(test_data_subset, train_data_subset)

# Display results
for result in results:
    print("Premise:", result["premise"])
    print("Hypothesis:", result["hypothesis"])
    print("Plug-in Prediction:", result["plugin_prediction"])
    print("Final Prediction:", result["final_prediction"])
    print("Explanation:", result["explanation"])
    print()

Step,Training Loss
10,1.110700
20,1.138700
30,1.107500
40,1.093500
50,1.104100
60,1.068000
70,1.087000
80,1.106800
90,1.113800
100,1.096000


Fine-tuning complete.
Premise: The new rights are nice enough
Hypothesis: Everyone really likes the newest benefits 
Plug-in Prediction: ('LABEL_0', 0.3695170283317566)
Final Prediction: LABEL_0
Explanation: LLM agrees with the plug-in prediction.

Premise: This site includes a list of all award winners and a searchable database of Government Executive articles.
Hypothesis: The Government Executive articles housed on the website are not able to be searched.
Plug-in Prediction: ('LABEL_2', 0.3743441700935364)
Final Prediction: LABEL_2
Explanation: LLM agrees with the plug-in prediction.

Premise: uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him
Hypothesis: I like him for the most part, but would still enjoy seeing someone beat him.
Plug-in Prediction: ('LABEL_0', 0.3841956555843353)
Final Prediction: LABEL_0
Explanation: LLM agrees with the plug-in prediction.

Premise: yeah i i think my favorite restaurant i

In [ ]:
# Calculate accuracy
def calculate_accuracy(results):
    correct = 0
    for result in results:
        final_pred_label = int(result["final_prediction"].split("_")[-1])
        if final_pred_label == result["label"]:
            correct += 1
    accuracy = correct / len(results)
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Calculate and display accuracy
calculate_accuracy(results)


Accuracy: 50.00%


#MRPC


In [ ]:
# Load MRPC dataset
mrpc_data = load_dataset("glue", "mrpc")
train_data = mrpc_data['train']
test_data = mrpc_data['validation']
# Load model and tokenizer
plugin_model_name = "roberta-base"
plugin_model = AutoModelForSequenceClassification.from_pretrained(plugin_model_name, num_labels=2)
plugin_tokenizer = AutoTokenizer.from_pretrained(plugin_model_name)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
plugin_model.to(device)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
# Preprocess data
def preprocess_function(examples):
    return plugin_tokenizer(
        examples['sentence1'], examples['sentence2'],
        truncation=True, padding="max_length", max_length=128
    )

# Apply preprocessing
train_dataset = train_data.map(preprocess_function, batched=True)
test_dataset = test_data.map(preprocess_function, batched=True)

# Convert datasets to PyTorch tensor format
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
# Define evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [ ]:
# Define training arguments with updated eval_strategy and matching save_strategy
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated parameter name
    save_strategy="epoch",  # Matches eval_strategy for load_best_model_at_end
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed precision for faster training
    report_to="none"  # Disable report (e.g., TensorBoard) for simplicity
)

# Initialize Trainer for fine-tuning
trainer = Trainer(
    model=plugin_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=plugin_tokenizer,
    compute_metrics=compute_metrics
)

# Clear GPU cache before training
torch.cuda.empty_cache()

# Fine-tune the model
trainer.train()


<ipython-input-14-5a895f16df81>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.562500,0.382468,0.857843,0.899654,0.869565,0.931900
2,0.238700,0.408291,0.867647,0.902878,0.906137,0.899642
3,0.355300,0.525009,0.867647,0.904594,0.891986,0.917563


TrainOutput(global_step=1377, training_loss=0.40081608858157003, metrics={'train_runtime': 231.9857, 'train_samples_per_second': 47.434, 'train_steps_per_second': 5.936, 'total_flos': 723818513295360.0, 'train_loss': 0.40081608858157003, 'epoch': 3.0})

In [ ]:
# Function to get predictions with confidence scores from the fine-tuned plug-in model
def get_plugin_predictions(texts):
    if isinstance(texts, dict):  # Single prediction case
        texts = [texts]

    inputs = plugin_tokenizer([f"{text['sentence1']} [SEP] {text['sentence2']}" for text in texts],
                              return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad():
        outputs = plugin_model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    result = []
    for prediction in predictions:
        confidence, label_id = torch.max(prediction, dim=-1)
        label = f"LABEL_{label_id.item()}"
        result.append((label, confidence.item()))

    return result

In [ ]:
# Prediction function using SuperICL with reduced context
def supericl_predict_mrpc(sentence1, sentence2, context, plugin_prediction):
    plugin_label, plugin_confidence = plugin_prediction

    if plugin_confidence >= 0.9:  # High confidence threshold
        return plugin_label, "Confidence is high; no LLM call needed."

    prompt = (
        f"{context}\n"
        f"Sentence 1: {sentence1}\n"
        f"Sentence 2: {sentence2}\n"
        f"Plug-in Prediction: {plugin_label} (Confidence: {plugin_confidence:.2f})\n"
        f"Are these sentences paraphrases of each other? "
        f"Provide 'LABEL_1' if yes, and 'LABEL_0' if no."
    )

    # Placeholder response simulating the LLM call
    llm_response = "LABEL_1" if "paraphrase" in prompt else plugin_label
    final_prediction = llm_response if "LABEL_" in llm_response else plugin_label

    if final_prediction != plugin_label:
        explanation = f"LLM overrides plug-in prediction from {plugin_label} to {final_prediction} based on context."
    else:
        explanation = "LLM agrees with plug-in prediction."

    return final_prediction, explanation

In [ ]:
import random

def run_supericl_mrpc(test_data, training_data, num_context_examples=1, max_test_examples=5):
    """
    Run SuperICL on test examples with reduced context for faster execution.

    Args:
    - test_data: List of test examples (sentence1, sentence2 pairs).
    - training_data: List of in-context examples.
    - num_context_examples: Number of in-context examples to include.
    - max_test_examples: Number of test examples to process.

    Returns:
    - A list of results with predictions and explanations.
    """
    # Sample in-context examples
    num_context_examples = min(num_context_examples, len(training_data))
    in_context_examples = random.sample(training_data, k=num_context_examples)

    # Prepare context with correct dictionary structure
    context = "\n".join([
        f"Sentence 1: {ex['sentence1']} Sentence 2: {ex['sentence2']}"
        for ex in in_context_examples
    ])

    # Process test examples
    results = []
    for example in test_data[:max_test_examples]:
        sentence1 = example["sentence1"]
        sentence2 = example["sentence2"]

        # Pass dictionaries with keys to get_plugin_predictions
        plugin_prediction = get_plugin_predictions([{"sentence1": sentence1, "sentence2": sentence2}])[0]

        # Run SuperICL for final prediction
        final_prediction, explanation = supericl_predict_mrpc(sentence1, sentence2, context, plugin_prediction)

        # Append result
        results.append({
            "sentence1": sentence1,
            "sentence2": sentence2,
            "plugin_prediction": plugin_prediction,
            "final_prediction": final_prediction,
            "explanation": explanation
        })

    return results


In [ ]:
# Convert datasets to lists for sampling
train_data_list = list(train_data)
test_data_list = list(test_data)

# Run and display results for only 5 sentences
results_mrpc = run_supericl_mrpc(test_data_list, train_data_list, max_test_examples=5)
for i, result in enumerate(results_mrpc, start=1):
    print(f"Example {i}:")
    print("Sentence 1:", result["sentence1"])
    print("Sentence 2:", result["sentence2"])
    print("Plugin Prediction:", result["plugin_prediction"])
    print("Final Prediction:", result["final_prediction"])
    print("Explanation:", result["explanation"])
    print()


Example 1:
Sentence 1: He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .
Sentence 2: " The foodservice pie business does not fit our long-term growth strategy .
Plugin Prediction: ('LABEL_1', 0.9860711693763733)
Final Prediction: LABEL_1
Explanation: Confidence is high; no LLM call needed.

Example 2:
Sentence 1: Magnarelli said Racicot hated the Iraqi regime and looked forward to using his long years of training in the war .
Sentence 2: His wife said he was " 100 percent behind George Bush " and looked forward to using his years of training in the war .
Plugin Prediction: ('LABEL_0', 0.9046505093574524)
Final Prediction: LABEL_0
Explanation: Confidence is high; no LLM call needed.

Example 3:
Sentence 1: The dollar was at 116.92 yen against the yen , flat on the session , and at 1.2891 against the Swiss franc , also flat .
Sentence 2: The dollar was at 116.78 yen JPY = , virtually flat on the session , and at 1.2871 against the Swiss franc CH

In [ ]:
# Define true labels for these examples (replace with actual labels in full dataset usage)
true_labels = ["LABEL_1", "LABEL_0", "LABEL_1", "LABEL_1", "LABEL_0"]  # Example true labels for demonstration

# Run SuperICL with reduced context on the test data
results_mrpc = run_supericl_mrpc(test_data_list, train_data_list, max_test_examples=5)

# Calculate accuracy
correct_predictions = 0
for i, result in enumerate(results_mrpc):
    final_prediction = result["final_prediction"]
    true_label = true_labels[i]
    if final_prediction == true_label:
        correct_predictions += 1

# Compute and print accuracy
accuracy = correct_predictions / len(results_mrpc)
print(f"SuperICL Accuracy: {accuracy * 100:.2f}%")



SuperICL Accuracy: 80.00%
